# WFABBA

WFABBA files are currently stored on an SDSC server as individual alert files. These files need to be downloaded and parsed into a table.

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import pathlib
import os
import time

In [2]:
metadataColNames = ["Algorithm", "Version", "Timestamp", "Satellite", "Instrument", "FlightModel", "ScanMode", "ProductType", "FileName", "DataSource",
               "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "MissingValueCode"]

dataColNames = ['Latitude', 'Longitude', 'Code', 'FRP', 'Fire Size', 'Fire Temp', 'Line', 'Element', 'Pixel Size',  'Obs BT4', 'Obs BT11', 'Bkg BT4', 'Bkg BT11', 'SolZen',  'SatZen', 'RelAzi', 'Eco', 'FD_X', 'FD_Y']

allColNames = metadataColNames + dataColNames

masterArr = []

In [3]:
starttime = time.time()
currentDir = os.path.abspath(os.getcwd())
# targetDir = currentDir + "/sdge.sdsc.edu/data/wfabba/goes-16"
# targetDir = currentDir + "/sdge.sdsc.edu/data/wfabba/goes-17"
# targetDir = currentDir + "/sdge.sdsc.edu/data/wfabba/goes-17/2021_01_20_020"
# targetDir = currentDir + "/sdge.sdsc.edu/data/wfabba/goes-17/2021_09_24_267"
# targetDir = currentDir + "/sdge.sdsc.edu/data/wfabba/goes-17/2021_01_31_031"
targetDir = currentDir + "/Temp2"
# count = 0

for root, subdirs, files in sorted(os.walk(targetDir)):
    print(root)
    
    if len(files) > 0:
        if files[0][-4:] == ".txt":
            for file in files:
                filepath = root+"/"+file
                if filepath[-4:] != ".txt":
                    print(filepath)
                    continue
                fo = open(filepath ,'r')
                metadataDict = {}
                dataArr = []
                lineCount = 0
                for line in fo:
                    # print(line)
                    if line[0:4] == "### ":
                        if lineCount < 11:
                            parseString = line[4:]
                            parseString = parseString.strip()

                            if parseString[0:4] == 'Date':
                                year = parseString[6:10]
                                day_num = parseString[10:13]
                                hr = parseString[21:23]
                                min = parseString[24:26]
                                sec = parseString[27:29]
                                res = dt.datetime.strptime(year + "-" + day_num + " " + hr + ":" + min + ":" + sec, "%Y-%j %H:%M:%S")
                                metadataDict["Timestamp"] = res
                            else:
                                splitData = parseString.split(", ")
                                # print(splitData)
                                for item in splitData:
                                    if item[-9:] == 'Algorithm':
                                        metadataDict["Algorithm"] = item
                                    else:
                                        sections = item.split(":")

                                        if sections[0] == 'Flight Model':
                                            keyname = 'FlightModel'
                                        elif sections[0] == 'Scan Mode':
                                            keyname = 'ScanMode'
                                        elif sections[0] == 'Product type':
                                            keyname = 'ProductType'
                                        elif sections[0] == 'Product/L2 filename':
                                            keyname = 'FileName'
                                        elif sections[0] == 'Data source':
                                            keyname = 'DataSource'
                                        elif sections[0] == 'Data creation time stamp':
                                            keyname = 'DataCreationTimestamp'
                                        elif sections[0] == 'Navigation projection subpoint longitude':
                                            keyname = 'NavProjSubPtLong'
                                        elif sections[0] == 'Actual satellite subpoint longitude':
                                            keyname = 'ActualSatSubPtLong'
                                        elif sections[0] == 'Number of detected fires':
                                            keyname = 'NumFire'
                                        elif sections[0] == 'Missing value code':
                                            keyname = 'MissingValueCode'
                                        else:
                                            keyname = sections[0]

                                        if len(sections) > 1:
                                            if sections[1] == '':
                                                value = None
                                            else:
                                                value = sections[1].strip()
                                            metadataDict[keyname] = value
                            lineCount+=1
                    elif line[0:3] != '###':
                        line_list = line.replace(' ', '').replace("\n","").split(',')
                        app_dict = {dataColNames[i]: line_list[i] for i in range(len(dataColNames))}
                        dataArr.append(app_dict)
                for dataDict in dataArr:
                    allDataDict = {}
                    allDataDict.update(metadataDict)
                    allDataDict.update(dataDict)
                    masterArr.append(allDataDict)
                fo.close()
                
df = pd.DataFrame(masterArr, columns=allColNames)
endtime = time.time()
print(endtime-starttime)

/home/ezen/Documents/DSE/2020-ezen/DSE260/Temp2
/home/ezen/Documents/DSE/2020-ezen/DSE260/Temp2/2021_09_01_244
/home/ezen/Documents/DSE/2020-ezen/DSE260/Temp2/2021_09_02_245
/home/ezen/Documents/DSE/2020-ezen/DSE260/Temp2/2021_09_03_246
/home/ezen/Documents/DSE/2020-ezen/DSE260/Temp2/2021_09_04_247
/home/ezen/Documents/DSE/2020-ezen/DSE260/Temp2/2021_09_05_248
/home/ezen/Documents/DSE/2020-ezen/DSE260/Temp2/2021_09_06_249
/home/ezen/Documents/DSE/2020-ezen/DSE260/Temp2/2021_09_07_250
/home/ezen/Documents/DSE/2020-ezen/DSE260/Temp2/2021_09_08_251
/home/ezen/Documents/DSE/2020-ezen/DSE260/Temp2/2021_09_09_252
/home/ezen/Documents/DSE/2020-ezen/DSE260/Temp2/2021_09_10_253
/home/ezen/Documents/DSE/2020-ezen/DSE260/Temp2/2021_09_11_254
/home/ezen/Documents/DSE/2020-ezen/DSE260/Temp2/2021_09_12_255
/home/ezen/Documents/DSE/2020-ezen/DSE260/Temp2/2021_09_13_256
/home/ezen/Documents/DSE/2020-ezen/DSE260/Temp2/2021_09_14_257
/home/ezen/Documents/DSE/2020-ezen/DSE260/Temp2/2021_09_15_258
/home/e

In [4]:
df

,Algorithm,Version,Timestamp,Satellite,Instrument,FlightModel,ScanMode,ProductType,FileName,DataSource,...,Obs BT4,Obs BT11,Bkg BT4,Bkg BT11,SolZen,SatZen,RelAzi,Eco,FD_X,FD_Y
0,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-09-01 21:55:55,GOES-17,ABI,FM?,GOES-17,GOES-17,f2021244215555.rev.6_6_001g.FDCM2.GOES-17.txt,SSEC,...,320.705,299.385,313.072,298.191,40.515,46.592,-9999,51,3683,1023
1,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-09-01 15:01:55,GOES-17,ABI,FM?,GOES-17,GOES-17,f2021244150155.rev.6_6_001g.FDCM2.GOES-17.txt,SSEC,...,296.889,282.224,291.465,287.582,66.147,48.541,-9999,51,3750,990
2,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-09-01 02:59:49,GOES-17,ABI,FM?,GOES-17,GOES-17,f2021244025949.rev.6_6_001g.FDCM2.GOES-17.txt,SSEC,...,288.654,277.415,276.942,276.753,101.378,47.130,-9999,51,3667,994
3,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-09-01 02:59:49,GOES-17,ABI,FM?,GOES-17,GOES-17,f2021244025949.rev.6_6_001g.FDCM2.GOES-17.txt,SSEC,...,288.397,279.886,276.670,276.479,101.398,47.145,-9999,51,3668,994
4,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-09-01 02:59:49,GOES-17,ABI,FM?,GOES-17,GOES-17,f2021244025949.rev.6_6_001g.FDCM2.GOES-17.txt,SSEC,...,291.910,283.636,276.998,276.799,101.380,47.103,-9999,51,3667,995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509989,WFABBA / Fire Detection and Characterization A...,6_6_001g,2022-02-10 00:40:31,GOES-17,ABI,FM?,GOES-17,GOES-17,f2022041004031.rev.6_6_001g.FDCF.GOES-17.txt,SSEC,...,301.612,290.239,291.898,291.449,81.850,43.385,-9999,22,3661,1134
509990,WFABBA / Fire Detection and Characterization A...,6_6_001g,2022-02-10 04:36:17,GOES-17,ABI,FM?,GOES-17,GOES-17,f2022041043617.rev.6_6_001g.FDCC.GOES-17.txt,SSEC,...,296.361,284.100,281.799,282.391,131.324,43.385,-9999,22,3661,1134
509991,WFABBA / Fire Detection and Characterization A...,6_6_001g,2022-02-10 00:20:31,GOES-17,ABI,FM?,GOES-17,GOES-17,f2022041002031.rev.6_6_001g.FDCF.GOES-17.txt,SSEC,...,308.342,292.378,294.698,293.493,77.944,43.385,-9999,22,3661,1134
509992,WFABBA / Fire Detection and Characterization A...,6_6_001g,2022-02-10 00:16:17,GOES-17,ABI,FM?,GOES-17,GOES-17,f2022041001617.rev.6_6_001g.FDCC.GOES-17.txt,SSEC,...,305.574,292.654,295.230,293.878,77.174,43.385,-9999,22,3661,1134


In [6]:
df.to_csv("goes-17_2.csv")